In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.4607591279782355 
Enpoint: people/475627 | Time: 0.46180694398935884 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.515759842004627 
PROJECTS
Enpoint: projects | Time: 0.7870564680197276 
ROLES
Enpoint: roles | Time: 0.5374683410045691 
PEOPLE
Enpoint: people | Time: 0.6035255580209196 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.4663430649670772 
MILESTONES
Enpoint: milestones | Time: 0.44327377196168527 
ASSIGNMENTS
Enpoint: assignments | Time: 0.5473153530037962 
DONE! (4.9s)


REG team
=====

We need a list of the team members full names from Forecast to compare with their names on GitHub. They match most of the time except when there is no GitHub name. Mapping below covers the rest of the team. We need to update this list if someone new is added to Forecast who doesn't have a matching GitHub name.

In [5]:
names = list(fc.people.full_name)
names.remove('Giovanni Colavizza')
names.remove('Miguel Morin')
names.remove('Mohammed Ali Al-Badri')
names

['Oliver Strickson',
 'Angus Williams',
 'David Beavan',
 'Evelina Gabasova',
 'James Geddes',
 'James Hetherington',
 'Louise Bowler',
 "Martin O'Reilly",
 'May Yong',
 'Nick Barlow',
 'Radka Jersakova',
 'Timothy Hobson',
 'Tomas Lazauskas',
 'Camila Rangel Smith',
 'Sarah Gibson',
 'James Robinson',
 'Eric Daub',
 'Jim Madge',
 'Amber Raza',
 'Kasra Hosseini',
 'Oscar Giles',
 'Joel Dearden',
 'Amaani Hoddoon',
 'Jack Roberts',
 'Oliver Forrest',
 'Gabriel Hanganu',
 'Ed Chalstrey',
 'Ashwini Venkatasubramaniam',
 'Roly Perera',
 'Federico Nanni',
 'Anna Hadjitofi',
 'Flora Roumpani']

Find a team member's availability for a date range
---

In [6]:
def get_person_availability(peopledf, person, start_date, end_date):
    """Get the mean of a person's FTE proportion available for the start to end datetime objects"""
    if isinstance(person, str):
        try:
            person = fc.get_person_id(person)
        except:
            return 0.0
    peopledf = peopledf[(peopledf.index >= start_date) & (peopledf.index <= end_date)]
    try:
        availability_range = peopledf[person]
    except:
        return 0.0
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)

In [7]:
peopledf = 1 - fc.people_totals.resample('MS').mean()  # pandas df for team members availability
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)
print(get_person_availability(peopledf, "Ed Chalstrey", start_date, end_date))
print(get_person_availability(peopledf, 458589, start_date, end_date))

0.65
1.0


Projects should have a consistent resource required number over the months they aren't 0, but may be useful to look at the resource requirement over a specific date range:

In [8]:
def get_project_requirement(projectdf, project, start_date, end_date):
    """Get the mean of a project's FTE requirement for the start to end datetime objects"""
    if isinstance(project, str):
        try:
            project = fc.get_project_id(project)
        except:
            return 0.0
    projectdf = projectdf[(projectdf.index >= start_date) & (projectdf.index <= end_date)]
    try:
        requirement_range = projectdf[project]
    except:
        return 0.0
    average_requirement = statistics.mean(requirement_range)
    return round(average_requirement, 2)

In [9]:
projectdf = fc.project_resourcereq.resample('MS').mean()  # pandas df for team members availability
print(get_project_requirement(projectdf, "CHANCE", start_date, end_date))
print(get_project_requirement(projectdf, 2061048, start_date, end_date))

0.62
0.23


Create queries with GraphQL to get the emojis for each GitHub issue
===

https://developer.github.com/v4/guides/forming-calls/

In [10]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

Check if a GitHub issue has emojis, based on the result of GraphQL query
----

In [11]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

In [12]:
def get_preference_data(fc):
    """Get each team members preference emoji for all projects with a GitHub issue"""
    issues = fc.projects["GitHub"].dropna()  # Get list of GitHub issues for projects
    gid_mapping = {  # People without their full names on github.
     'myyong': 'May Yong',
     'nbarlowATI': 'Nick Barlow',
     'thobson88': 'Timothy Hobson',
     'miguelmorin': 'Miguel Morin',
     'OscartGiles': 'Oscar Giles',
     'AshwiniKV': 'Ashwini Venkatasubramaniam',
    }
    emoji_mapping = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}
    names = list(fc.people.full_name)
    names.remove('Giovanni Colavizza')
    names.remove('Miguel Morin')
    names.remove('Mohammed Ali Al-Badri')
    preference_data = {
        "Person": names
    }
    for issue_num, project_id in zip(issues, issues.index):
        modified_query = query.replace("X", str(issue_num))
        result = run_query(modified_query)  # Execute the query
#         if query_result_contains_emojis(result)  # Could do this if we only want issues with emojis
        emojis = []
        for name in names: 
            emoji_name = None
            for reaction in result['data']['repository']['issue']['reactionGroups']:
                for edge in reaction['users']['edges']:
                    if edge['node']['name'] == name:
                        emoji_name = reaction['content']
                    if not emoji_name:
                        if edge['node']['login'] in gid_mapping:
                            if gid_mapping[edge['node']['login']] == name:
                                emoji_name = reaction['content']
            if emoji_name:
                emoji = emoji_mapping[emoji_name]
            else:
                emoji = "❓"
            emojis.append(emoji)
        preference_data[fc.get_project_name(project_id)] = emojis
    return preference_data

In [13]:
preference_data = get_preference_data(fc)

In [64]:
preference_data_df = pd.DataFrame(preference_data).set_index('Person')[(preferences.T != '❓').any()]
preference_data_df

,Sargasso (Intel Sparse),DetectorChecker,HSBC EDS,AI for City Planning,Reproducible Research,PDQ,Crypto Federated Learning (SHEEP),The Turing Way,Fitbit classification,Parallel Monte Carlo,...,Interpretable models,Census 2021 backbrief,Solar Panel Detection,NoCell - Phase 2,Hate Speech,SAIL multimorbidity,QUIPP - Synthetic data,WAYS WP1 - Extending BackFillz,WAYS WP2 - Auto-generating precursor visualisations,Survival outcome - Roche
Person,,,,,,,,,,,,,,,,,,,,,
Oliver Strickson,❓,❓,❓,❓,❓,❓,❓,❌,❓,❓,...,❓,👍,👍,✅,❌,👍,👍,👍,👍,❓
Evelina Gabasova,❓,❓,❓,❓,❓,❓,❓,✅,👍,👍,...,❓,❓,👍,👍,👍,👍,👍,👍,👍,❓
James Geddes,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,👍,❓,✅,❓,❓,❓,❓,❓,❓
James Hetherington,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
Louise Bowler,❓,❓,❓,❓,❓,❓,❓,✅,❌,❌,...,❓,👍,👍,❌,👍,👍,👍,👍,👍,❓
May Yong,❓,❓,❓,❓,❓,❓,❓,❌,❓,❓,...,❓,❌,❓,❌,👍,❓,❌,👍,👍,❓
Nick Barlow,❓,❓,❓,❓,❓,❓,❓,❌,👍,❌,...,❓,❌,👍,👍,👍,👍,👍,👍,👍,❓
Radka Jersakova,❓,❓,❓,❓,❓,❓,❓,👍,❌,❌,...,❓,👍,❓,❌,👍,👍,👍,❌,❌,❓
Timothy Hobson,❓,❓,❓,❓,❓,❓,❓,❌,❌,👍,...,❓,👍,👍,👍,❌,👍,👍,❌,❌,❓


In [65]:
preference_data_df["CHANCE"]["Ed Chalstrey"]

'✅'

Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [66]:
def get_preferences(fc, preference_data_df, first_date=False, last_date=False, person=False, project=False, positive_only=False, emojis_only=False):
    resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
    if person:
        names = [person]
    else:
        names = list(preference_data_df.index)
    data = {
        "Person": names
    }
    issues = fc.projects["GitHub"].dropna().values
    peopledf = 1 - fc.people_totals.resample('MS').mean()
    projectdf = fc.project_resourcereq.resample('MS').mean()  # pandas df for team members availability
    if project:
        if isinstance(project, str):
            try:
                project = fc.get_project_id(project)
            except:
                pass
    for project_id in resreqdf:  # some of these have no GitHub issue
        if not project or project == project_id:
            date_indices = resreqdf.index[resreqdf[project_id] > 0]
            if len(date_indices) > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
                issue_num = fc.projects.loc[project_id, "GitHub"]
                if issue_num in issues:  # if this project has a GitHub issue
    #                 start_date = fc.projects.loc[project_id, "start_date"]
    #                 end_date = fc.projects.loc[project_id, "end_date"]
                    first_resreq_date = date_indices[0].strftime("%Y-%m-%d")
                    last_resreq_date = date_indices[-1].strftime("%Y-%m-%d")
                    resreq = get_project_requirement(projectdf, project_id, first_resreq_date, last_resreq_date)
                    project_title = fc.projects.loc[project_id, "name"]
                    emoji_data = []
                    for name in names:
#                         print(name)
                        person_availability = get_person_availability(peopledf, name, first_resreq_date, last_resreq_date)
                        percentage_availability = round((person_availability / resreq) * 100, 2)
                        emoji = preference_data_df[project_title][name]
                        if (not person and not project) or not positive_only or emoji == '✅' or emoji == '👍':
                            if emojis_only:
                                emoji_data.append(emoji)
                            else:
                                emoji_data.append(emoji + " " + str(percentage_availability) + "% (" + str(person_availability) + " / " + str(round(resreq, 2)) + ")")
#                         if project and positive_only and (emoji == '❌' or emoji == '❓'):
#                             print(name, emoji)
#                             data["Person"].remove(name)
#                         else:
#                             print(name, emoji)
                            
                    if (not person and not project) or not positive_only or len(emoji_data) > 0:
                        data[project_title] = emoji_data
                        
#     print(data)
    preferences = pd.DataFrame(data).set_index('Person').sort_index().sort_index(axis=1)
    return preferences

All projects with team members preferences and their average availability between the start of project resource requirement and project end date
----

In [67]:
preferences = get_preferences(fc, preference_data_df, emojis_only=True)

In [75]:
preferences

,All in One Cancer imaging optimisation,Auto DL,CHANCE,Census 2021 backbrief,DFTG Commons WP 3.4/3.5,DFTG Commons scoping,Interpretable AI for health,Learning Machines,Living With Machines,Logics for DS,...,Probabilistic FEM,SAIL multimorbidity,Safe Haven,Scaleable Monte Carlo,Solar Panel Detection,UQM^3,Uncertainty in Government Modelling,Urban systems resilience,Visualisation: Roy Ruddle TF project,Wrattler 2019-2020
Person,,,,,,,,,,,,,,,,,,,,,
Ashwini Venkatasubramaniam,👍,❌,❌,❌,❌,❌,✅,❓,❓,❌,...,❓,👍,❓,❌,❓,❓,❓,❓,❌,❓
Camila Rangel Smith,👍,✅,❓,👍,❌,❌,👍,❓,👍,👍,...,❌,👍,❓,❌,👍,👍,👍,👍,❓,❓
Ed Chalstrey,✅,❌,✅,👍,❓,👍,👍,❓,❓,❌,...,❓,👍,❓,👍,✅,❓,❓,❌,❌,❓
Eric Daub,❌,❌,👍,❓,👍,👍,❌,❓,❓,❌,...,❓,❌,❓,👍,👍,❓,❓,👍,❌,❓
Evelina Gabasova,👍,👍,👍,❓,❌,❌,👍,❓,👍,👍,...,❌,👍,❓,❌,👍,❌,👍,👍,❌,❓
Jack Roberts,👍,👍,👍,👍,❌,❌,👍,❓,✅,❌,...,👍,👍,❓,❓,✅,❌,❓,👍,❌,❓
James Geddes,❓,❓,❓,👍,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,👍,❓,❓,❓,❓,❓,❓
James Hetherington,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,✅,❓,❓,❓,❓,❓,❓,❓,❓,❓
James Robinson,❌,✅,👍,👍,❌,❌,👍,❓,👍,❌,...,❌,👍,❌,✅,👍,👍,👍,👍,❌,❓


In [69]:
emoji_table = preferences.to_html()
emoji_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>All in One Cancer imaging optimisation</th>\n      <th>Auto DL</th>\n      <th>CHANCE</th>\n      <th>Census 2021 backbrief</th>\n      <th>DFTG Commons WP 3.4/3.5</th>\n      <th>DFTG Commons scoping</th>\n      <th>Interpretable AI for health</th>\n      <th>Learning Machines</th>\n      <th>Living With Machines</th>\n      <th>Logics for DS</th>\n      <th>Machine learning for cancer</th>\n      <th>Micro-location analytics</th>\n      <th>NoCell - Phase 2</th>\n      <th>Probabilistic FEM</th>\n      <th>SAIL multimorbidity</th>\n      <th>Safe Haven</th>\n      <th>Scaleable Monte Carlo</th>\n      <th>Solar Panel Detection</th>\n      <th>UQM^3</th>\n      <th>Uncertainty in Government Modelling</th>\n      <th>Urban systems resilience</th>\n      <th>Visualisation: Roy Ruddle TF project</th>\n      <th>Wrattler 2019-2020</th>\n    </tr>\n    <tr>\n      <th>Person</t

In [70]:
preferences_with_availability = get_preferences(fc, preference_data_df)

In [71]:
preferences_with_availability

,All in One Cancer imaging optimisation,Auto DL,CHANCE,Census 2021 backbrief,DFTG Commons WP 3.4/3.5,DFTG Commons scoping,Interpretable AI for health,Learning Machines,Living With Machines,Logics for DS,...,Probabilistic FEM,SAIL multimorbidity,Safe Haven,Scaleable Monte Carlo,Solar Panel Detection,UQM^3,Uncertainty in Government Modelling,Urban systems resilience,Visualisation: Roy Ruddle TF project,Wrattler 2019-2020
Person,,,,,,,,,,,,,,,,,,,,,
Ashwini Venkatasubramaniam,👍 42.0% (0.42 / 1.0),❌ 84.0% (0.42 / 0.5),❌ 94.0% (0.94 / 1.0),❌ 89.29% (0.25 / 0.28),❌ 14.5% (0.58 / 4.0),❌ 62.0% (0.62 / 1.0),✅ 100.0% (1.0 / 1.0),❓ 51.89% (0.96 / 1.85),❓ 51.39% (0.37 / 0.72),❌ 264.0% (0.66 / 0.25),...,❓ 180.0% (0.9 / 0.5),👍 0.0% (0.0 / 0.5),❓ 28.79% (0.38 / 1.32),❌ 8.0% (0.08 / 1.0),❓ 184.0% (0.92 / 0.5),❓ 200.0% (1.0 / 0.5),❓ 122.0% (0.61 / 0.5),❓ 83.0% (0.83 / 1.0),❌ 79.0% (0.79 / 1.0),❓ 100.0% (1.0 / 1.0)
Camila Rangel Smith,👍 100.0% (1.0 / 1.0),✅ 16.0% (0.08 / 0.5),❓ 69.0% (0.69 / 1.0),👍 0.0% (0.0 / 0.28),❌ 25.0% (1.0 / 4.0),❌ 12.0% (0.12 / 1.0),👍 100.0% (1.0 / 1.0),❓ 43.78% (0.81 / 1.85),👍 138.89% (1.0 / 0.72),👍 264.0% (0.66 / 0.25),...,❌ 100.0% (0.5 / 0.5),👍 200.0% (1.0 / 0.5),❓ 18.94% (0.25 / 1.32),❌ 100.0% (1.0 / 1.0),👍 116.0% (0.58 / 0.5),👍 200.0% (1.0 / 0.5),👍 78.0% (0.39 / 0.5),👍 17.0% (0.17 / 1.0),❓ 50.0% (0.5 / 1.0),❓ 100.0% (1.0 / 1.0)
Ed Chalstrey,✅ 29.0% (0.29 / 1.0),❌ 66.0% (0.33 / 0.5),✅ 81.0% (0.81 / 1.0),👍 178.57% (0.5 / 0.28),❓ 8.25% (0.33 / 4.0),👍 50.0% (0.5 / 1.0),👍 100.0% (1.0 / 1.0),❓ 35.14% (0.65 / 1.85),❓ 36.11% (0.26 / 0.72),❌ 188.0% (0.47 / 0.25),...,❓ 140.0% (0.7 / 0.5),👍 0.0% (0.0 / 0.5),❓ 27.27% (0.36 / 1.32),👍 0.0% (0.0 / 1.0),✅ 150.0% (0.75 / 0.5),❓ 172.0% (0.86 / 0.5),❓ 112.0% (0.56 / 0.5),❌ 50.0% (0.5 / 1.0),❌ 71.0% (0.71 / 1.0),❓ 100.0% (1.0 / 1.0)
Eric Daub,❌ 83.0% (0.83 / 1.0),❌ 0.0% (0.0 / 0.5),👍 31.0% (0.31 / 1.0),❓ 0.0% (0.0 / 0.28),👍 19.75% (0.79 / 4.0),👍 0.0% (0.0 / 1.0),❌ 50.0% (0.5 / 1.0),❓ 24.86% (0.46 / 1.85),❓ 118.06% (0.85 / 0.72),❌ 168.0% (0.42 / 0.25),...,❓ 40.0% (0.2 / 0.5),❌ 200.0% (1.0 / 0.5),❓ 8.33% (0.11 / 1.32),👍 100.0% (1.0 / 1.0),👍 50.0% (0.25 / 0.5),❓ 100.0% (0.5 / 0.5),❓ 34.0% (0.17 / 0.5),👍 0.0% (0.0 / 1.0),❌ 21.0% (0.21 / 1.0),❓ 50.0% (0.5 / 1.0)
Evelina Gabasova,👍 50.0% (0.5 / 1.0),👍 0.0% (0.0 / 0.5),👍 31.0% (0.31 / 1.0),❓ 0.0% (0.0 / 0.28),❌ 12.5% (0.5 / 4.0),❌ 0.0% (0.0 / 1.0),👍 50.0% (0.5 / 1.0),❓ 20.54% (0.38 / 1.85),👍 69.44% (0.5 / 0.72),👍 128.0% (0.32 / 0.25),...,❌ 40.0% (0.2 / 0.5),👍 100.0% (0.5 / 0.5),❓ 8.33% (0.11 / 1.32),❌ 50.0% (0.5 / 1.0),👍 50.0% (0.25 / 0.5),❌ 100.0% (0.5 / 0.5),👍 34.0% (0.17 / 0.5),👍 0.0% (0.0 / 1.0),❌ 21.0% (0.21 / 1.0),❓ 50.0% (0.5 / 1.0)
Jack Roberts,👍 67.0% (0.67 / 1.0),👍 66.0% (0.33 / 0.5),👍 50.0% (0.5 / 1.0),👍 178.57% (0.5 / 0.28),❌ 12.5% (0.5 / 4.0),❌ 50.0% (0.5 / 1.0),👍 50.0% (0.5 / 1.0),❓ 27.03% (0.5 / 1.85),✅ 97.22% (0.7 / 0.72),❌ 168.0% (0.42 / 0.25),...,👍 100.0% (0.5 / 0.5),👍 134.0% (0.67 / 0.5),❓ 18.94% (0.25 / 1.32),❓ 71.0% (0.71 / 1.0),✅ 100.0% (0.5 / 0.5),❌ 100.0% (0.5 / 0.5),❓ 78.0% (0.39 / 0.5),👍 50.0% (0.5 / 1.0),❌ 50.0% (0.5 / 1.0),❓ 50.0% (0.5 / 1.0)
James Geddes,❓ 47.0% (0.47 / 1.0),❓ 4.0% (0.02 / 0.5),❓ 26.0% (0.26 / 1.0),👍 0.0% (0.0 / 0.28),❓ 8.75% (0.35 / 4.0),❓ 2.0% (0.02 / 1.0),❓ 10.0% (0.1 / 1.0),❓ 12.97% (0.24 / 1.85),❓ 68.06% (0.49 / 0.72),❓ 92.0% (0.23 / 0.25),...,❓ 52.0% (0.26 / 0.5),❓ 120.0% (0.6 / 0.5),❓ 10.61% (0.14 / 1.32),👍 60.0% (0.6 / 1.0),❓ 64.0% (0.32 / 0.5),❓ 48.0% (0.24 / 0.5),❓ 42.0% (0.21 / 0.5),❓ 3.0% (0.03 / 1.0),❓ 27.0% (0.27 / 1.0),❓ 60.0% (0.6 / 1.0)
James Hetherington,❓ 9.0% (0.09 / 1.0),❓ 0.0% (0.0 / 0.5),❓ 2.0% (0.02 / 1.0),❓ 0.0% (0.0 / 0.28),❓ 2.5% (0.1 / 4.0),❓ 0.0% (0.0 / 1.0),❓ 10.0% (0.1 / 1.0),❓ 2.7% (0.05 / 1.85),❓ 12.5% (0.09 / 0.72),❓ 20.0% (0.05 / 0.25),...,✅ 0.0% (0.0 / 0.5),❓ 20.0% (0.1 / 0.5),❓ 0.0% (0.0 / 1.32),❓ 10.0% (0.1 / 1.0),❓ 0.0% (0.0 / 0.5),❓ 14.0% (0.07 / 0.5),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 1.0)
James Robinson,❌ 100.0% (1.0 / 1

In [72]:
get_preferences(fc, preference_data_df, person="Ed Chalstrey", positive_only=True)

,All in One Cancer imaging optimisation,CHANCE,Census 2021 backbrief,DFTG Commons scoping,Interpretable AI for health,Machine learning for cancer,Micro-location analytics,SAIL multimorbidity,Scaleable Monte Carlo,Solar Panel Detection
Person,,,,,,,,,,
Ed Chalstrey,✅ 29.0% (0.29 / 1.0),✅ 81.0% (0.81 / 1.0),👍 178.57% (0.5 / 0.28),👍 50.0% (0.5 / 1.0),👍 100.0% (1.0 / 1.0),✅ 79.0% (0.79 / 1.0),✅ 12.0% (0.12 / 1.0),👍 0.0% (0.0 / 0.5),👍 0.0% (0.0 / 1.0),✅ 150.0% (0.75 / 0.5)


In [73]:
get_preferences(fc, preference_data_df, project="CHANCE")

,CHANCE
Person,
Ashwini Venkatasubramaniam,❌ 94.0% (0.94 / 1.0)
Camila Rangel Smith,❓ 69.0% (0.69 / 1.0)
Ed Chalstrey,✅ 81.0% (0.81 / 1.0)
Eric Daub,👍 31.0% (0.31 / 1.0)
Evelina Gabasova,👍 31.0% (0.31 / 1.0)
Jack Roberts,👍 50.0% (0.5 / 1.0)
James Geddes,❓ 26.0% (0.26 / 1.0)
James Hetherington,❓ 2.0% (0.02 / 1.0)
James Robinson,👍 82.0% (0.82 / 1.0)


In [74]:
get_preferences(fc, preference_data_df, project="CHANCE", positive_only=True)

ValueError: arrays must all be same length